<a href="https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Arboles/Clase_02_Arboles/02_C_%C3%81rboles_Decisi%C3%B3n_sin_Overfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 INTRO: Árboles de Decisión
Explicación de cómo construír árboles de decisión.

Notebook por [Javier Blanco Cordero](https://www.linkedin.com/in/javier-blanco-cordero-71373656/).

### Enlaces de interés
*   [Slides de presentación](https://docs.google.com/presentation/d/1iq5k6zECRldUv5so26OsvkExqFJUwKd_CZu18hB-MB8/edit#slide=id.gc2469ceed5_0_0)



## 0101 Qué es un árbol de decisión?
Un tipo de algoritmo de aprendizaje supervisado que se basa en realizar particiones a partir de distintos niveles de las variables disponibles.

## 0102 Import
Importamos todas las librerías necesarias para este análisis ([¿No sabes lo que es una librería de Python?](https://www.quora.com/What-is-a-Python-library-and-what-can-I-use-it-for)): pandas, numpy, seaborn, matplotlib.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz 
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

## 0103 Carga el dataset de coches de segunda mano
Para probar a hacer árboles de decisión sin overfitting utilizaremos un dataset sobre el precio de distintos coches de segunda mano que he encontrado en Kaggle ([aquí](https://www.kaggle.com/harturo123/online-adds-of-used-cars)). 

Podéis encontrar el archivo listo para importar en mi github: 'https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/coches_usados_esp.csv'. 

Importa este dataset en un dataframe llamado **df**.

In [ ]:
# Url archivo raw
url = 'https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/coches_usados_esp.csv'

# Importa csv
df = pd.read_csv(url, sep=';')

# Visualización primeras filas
df.head()

# 02 EDA
Realizaremos un pequeño análisis exploratorio visual para familiarizarnos con el dataset. 

Recuerda que puedes encontrar mis clases sobre análisis exploratorio [aquí](https://github.com/JotaBlanco/TheValley/tree/main/EDA/).

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
# Visualización coeficientes Pearson
plt.figure(figsize=(8,7))
sns.heatmap(np.round(df.corr(),2), 
            vmin=-1, vmax=1, 
            annot=True, cmap="coolwarm")
plt.show()

# 03 EJEMPLO
Vamos a ver paso a paso cómo realizar un modelo que prediga el precio sin caer en overfitting.

## 0301 Preparamos los datos
El dataframe tiene algunos nulos, así como variables categóricas y presencia de ciertas variables que probablemente no queramos usar.

### 030101 Variables Útiles
De entre las variables disponibles, veamos cuáles queremos utilizar como predictoras para el estudio.

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
cols = ['make', 'model', 'months_old', 'power', 'sale_type', 'num_owners', 
        'gear_type', 'fuel_type', 'kms', 'price']

### 030102 Dumificación de variables categóricas

In [ ]:
df_i = pd.get_dummies(df[cols], 
                   prefix_sep='_', 
                   drop_first=True, 
                   columns=['make', 'model', 'sale_type', 'gear_type', 'fuel_type'])

display(len(df_i))
df_i.head()

### 030103 Limpieza de nulos
Con la dumificación hemos eliminado los nulos en las columnas categóricas sin deshacernos de las filas. Queda algún nulo en las variables numéricas?


In [ ]:
df_i.columns

In [ ]:
df_i[['months_old', 'power', 'num_owners', 'kms', 'price']].isna().sum()

In [ ]:
# Hay muchos nulos en num_owners
# Quizás esto tiene que ver con origenes del coche desconocidos?
# Vamos a limpiar la variable en 1, 2, 3+, nulo y la utilizamos como categórica
filtro_muchos_owners = df_i['num_owners']>=3
df_i.loc[filtro_muchos_owners, 'num_owners'] = '3+'
df_i = pd.get_dummies(df_i, prefix_sep='_', 
                   dummy_na=True,
                   drop_first=True, 
                   columns=['num_owners'])
df_i.head()

In [ ]:
df_i[['months_old', 'power', 'kms', 'price']].isna().sum()

In [ ]:
for col in ['months_old', 'power', 'kms']:
  df_i[col] = df_i[col].fillna(df_i[col].median())

In [ ]:
df_i.isna().sum()

##0302 Train - test split
Separamos el set de datos en dos utilizando [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).


In [ ]:
X = df_i.drop('price',axis=1)
y = df_i['price']

len(X), len(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## 0303 Entrenamos el árbol de decisión
Sobre el set de entrenamiento, comprobamos el modelo sobre el set de test.
https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor

In [ ]:
# Inicializo un árbol
modelo = tree.DecisionTreeRegressor(max_depth=10)
# Entreno el árbol con el set de entrenamiento
modelo = modelo.fit(X=X_train, y=y_train)
# Uso el árbol para predecir sobre el dataset de entrenamiento
y_pred_train = modelo.predict(X_train)
# Uso el árbol para predecir sobre el dataset de test
y_pred_test = modelo.predict(X_test)
# Cómo de buena es la predicción?
print('RMSE en set de entrenamiento :', mean_squared_error(y_train, y_pred_train, squared=False))
print('RMSE en set de test :', mean_squared_error(y_test, y_pred_test, squared=False))

In [ ]:
# Inicializo un árbol
modelo = tree.DecisionTreeRegressor(max_depth=5)
# Entreno el árbol con el set de entrenamiento
modelo = modelo.fit(X=X_train, y=y_train)
# Uso el árbol para predecir sobre el dataset de entrenamiento
y_pred_train = modelo.predict(X_train)
# Uso el árbol para predecir sobre el dataset de test
y_pred_test = modelo.predict(X_test)
# Cómo de buena es la predicción?
print('RMSE en set de entrenamiento :', mean_squared_error(y_train, y_pred_train, squared=False))
print('RMSE en set de test :', mean_squared_error(y_test, y_pred_test, squared=False))

In [ ]:
# Inicializo un árbol
modelo = tree.DecisionTreeRegressor(max_depth=50)
# Entreno el árbol con el set de entrenamiento
modelo = modelo.fit(X=X_train, y=y_train)
# Uso el árbol para predecir sobre el dataset de entrenamiento
y_pred_train = modelo.predict(X_train)
# Uso el árbol para predecir sobre el dataset de test
y_pred_test = modelo.predict(X_test)
# Cómo de buena es la predicción?
print('RMSE en set de entrenamiento :', mean_squared_error(y_train, y_pred_train, squared=False))
print('RMSE en set de test :', mean_squared_error(y_test, y_pred_test, squared=False))

In [ ]:
def entrenar_modelo_y_predecir(modelo):
  # Entreno el árbol con el set de entrenamiento
  modelo = modelo.fit(X=X_train, y=y_train)
  # Uso el árbol para predecir sobre el dataset de entrenamiento
  y_pred_train = modelo.predict(X_train)
  # Uso el árbol para predecir sobre el dataset de test
  y_pred_test = modelo.predict(X_test)
  # Cómo de buena es la predicción?
  print('RMSE en set de entrenamiento :', mean_squared_error(y_train, y_pred_train, squared=False))
  print('RMSE en set de test :', mean_squared_error(y_test, y_pred_test, squared=False))

In [ ]:
# Inicializo un árbol
modelo = tree.DecisionTreeRegressor(max_depth=25)
# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir(modelo)

## 0304 Probamos medidas contra el overfitting

### 030401 min_samples_split
Tamaño muestral mínimo para permitir una partición.

In [ ]:
# Inicializo un árbol
modelo = tree.DecisionTreeRegressor(max_depth=10, min_samples_split=20)
# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir(modelo)

In [ ]:
# Inicializo un árbol
modelo = tree.DecisionTreeRegressor(max_depth=15, min_samples_split=20)
# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir(modelo)

In [ ]:
# Inicializo un árbol
modelo = tree.DecisionTreeRegressor(max_depth=15, min_samples_split=25)
# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir(modelo)

### 030402 min_samples_leaf
Tamaño muestral mínimo en una hoja.

In [ ]:
# Inicializo un árbol
modelo = tree.DecisionTreeRegressor(max_depth=15, 
                                    min_samples_split=25,
                                    min_samples_leaf = 10)
# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir(modelo)

In [ ]:
# Inicializo un árbol
modelo = tree.DecisionTreeRegressor(max_depth=20, 
                                    min_samples_split=20,
                                    min_samples_leaf = 2)
# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir(modelo)

### 030402 min_samples_leaf
Mínimo descenso de impuridad.

In [ ]:
# Inicializo un árbol
modelo = tree.DecisionTreeRegressor(max_depth=15, 
                                    min_samples_split=25,
                                    min_impurity_decrease = 0.25)
# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir(modelo)

# 06 EJERCICIO TITANIC
Recordais el dataset del Titanic?

Vamos a resolver este problema teniendo en cuenta todo lo que sabemos ya. El objetivo es crear una árbol de decisión que prediga si un pasajero falleció o no (pasajeros cuyos datos no conocemos todavía).

Toma las medidas oportunas para que tu modelo sea lo más preciso posible sin caer en overfitting.

## 0601 Importa el dataset
Puedes encontrarlo en mi github. Este es el link al archivo raw: https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/titanic.csv.

Importa los datos en un dataframe llamado dataframe **df_titanic**.

In [ ]:
# Url archivo raw
url = 'https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/titanic.csv'

# Importa csv
df_titanic = pd.read_csv(url)

# Visualización primeras filas
df_titanic.head(2)

In [ ]:
df_titanic.columns

## 0602 Prepara los datos
Quédate con las variables interesantes, dumifica las categóricas y limpia los nulos.

## 0603 Train - test split
Utiliza una partición del 30% para tu set de pruebas.

## 0404 Entrena varios árboles
Entrena varios árboles de decisión controlando los distintos parámetros para buscar el punto óptimo entre bias y varianza.